In [1]:
import openml
from sklearn import preprocessing
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt

import torch
import sklearn

from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import datasets, transforms
from ucimlrepo import fetch_ucirepo 

import utils.losses as losses

In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

import utils.losses as losses
# Load Iris dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Labels

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_train_tensor = torch.eye(3)[y_train_tensor] 
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define Logistic Regression model for Multiclass Classification
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)  # Softmax will be applied by the loss function

# Initialize model, loss function, and optimizer
input_dim = X_train.shape[1]
output_dim = len(iris.target_names)  # Number of classes
model = LogisticRegressionModel(input_dim=input_dim, output_dim=output_dim)
loss_fn = losses.FwdLoss(torch.eye(3))
#criterion = nn.CrossEntropyLoss()  # Cross-Entropy Loss for multiclass classification
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        # Forward pass
        y_batch = torch.max(y_batch,dim=1).indices
        outputs = model(X_batch).to(device)
        print(outputs)
        print(y_batch)
        loss,Mp,z = loss_fn(outputs, y_batch.to(device))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
with torch.no_grad():
    y_pred_train = model(X_train_tensor)
    y_pred_test = model(X_test_tensor)

    # Convert logits to class labels
    _, y_pred_train = torch.max(y_pred_train, 1)
    _, y_pred_test = torch.max(y_pred_test, 1)

    # Calculate accuracy
    train_accuracy = (y_pred_train == y_train_tensor).float().mean()
    test_accuracy = (y_pred_test == y_test_tensor).float().mean()

    print(f'Train Accuracy: {train_accuracy:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')


c:\Users\danib\VSprojects\ForwardBackard_losses\utils\losses.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.M = torch.tensor(M, dtype=torch.float32, device=device)


tensor([[-1.0004,  0.9539, -0.2323],
        [-1.6134,  0.4867,  0.4534],
        [ 0.9810, -0.1354,  0.3386],
        [-1.0748,  0.9451, -0.1605],
        [-0.0107,  0.4212, -0.2013],
        [ 1.2680, -0.2674, -0.0358],
        [-1.5818,  0.5448,  0.3425],
        [-1.4912,  0.4040,  0.4957],
        [-0.7380, -0.2345,  0.7433],
        [-1.1654,  0.0842,  0.6868],
        [ 1.1991, -0.1981,  0.3438],
        [-0.0768,  0.2130,  0.2019],
        [-0.5604,  1.0024, -0.3926],
        [ 0.1147,  0.0500,  0.3928],
        [-0.2036,  0.3479,  0.0650],
        [-0.9503,  1.0625, -0.4224]], device='cuda:0',
       grad_fn=<ToCopyBackward0>)
tensor([1, 0, 2, 1, 1, 2, 0, 0, 0, 0, 2, 1, 2, 1, 1, 1])
tensor([[-0.1322,  0.5988, -0.1344],
        [ 0.9858, -0.3252,  0.5799],
        [ 0.4653,  0.1160,  0.0486],
        [-0.2590,  0.4482, -0.0502],
        [-0.0491,  0.3205,  0.1261],
        [-0.8440,  0.1667,  0.1022],
        [ 1.0856, -0.1175, -0.1440],
        [ 0.4092,  0.1519, -0.1061],
   

RuntimeError: The size of tensor a (105) must match the size of tensor b (3) at non-singleton dimension 1

In [11]:
import numpy as np
M = np.array([[1,0,0,0,0,0,0,0,0,0],
[0,1,0,0,0,0,0,0,0,0],
[0,0,.3,0,0,0,0,.7,0,0],
[0,0,0,.3,0,0,0,0,.7,0],
[0,0,0,0,1,0,0,0,0,0],
[0,0,0,0,0,.3,.7,0,0,0],
[0,0,0,0,0,.7,.3,0,0,0],
[0,.7,0,0,0,0,0,.3,0,0],
[0,0,0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,0,0,1]])

In [13]:
M.T

array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0.7, 0. , 0. ],
       [0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.3, 0.7, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.7, 0.3, 0. , 0. , 0. ],
       [0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0.3, 0. , 0. ],
       [0. , 0. , 0. , 0.7, 0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]])

In [7]:
print(Mp.shape)
print(Mp)

torch.Size([3, 9])
tensor([[0.1443, 0.1672, 0.0695, 0.1600, 0.0478, 0.2675, 0.2447, 0.6346, 0.2174],
        [0.7012, 0.3100, 0.5758, 0.2229, 0.8619, 0.4291, 0.3903, 0.1794, 0.4210],
        [0.1545, 0.5228, 0.3547, 0.6171, 0.0903, 0.3034, 0.3651, 0.1860, 0.3616]],
       device='cuda:0', grad_fn=<MmBackward0>)


In [8]:
z

tensor([1, 2, 1, 2, 1, 1, 1, 0, 2], device='cuda:0')

In [9]:
Mp[z,range(Mp.size(1))]

tensor([0.7012, 0.5228, 0.5758, 0.6171, 0.8619, 0.4291, 0.3903, 0.6346, 0.3616],
       device='cuda:0', grad_fn=<IndexBackward0>)

In [19]:
range(Mp.size(1))

range(0, 3)

In [7]:
y_batch = y_batch.long()

In [ ]:
softmax = torch.nn.Softmax(dim=1)

In [9]:
Mp = torch.tensor([[0.2607, 0.3270, 0.1814, 0.2682, 0.5227, 0.2219, 0.1594, 0.2619, 0.5241,
         0.2541, 0.0906, 0.3955, 0.2038, 0.5019, 0.1821, 0.2850],
        [0.3531, 0.3812, 0.4106, 0.4171, 0.3218, 0.4037, 0.3968, 0.3703, 0.3340,
         0.4167, 0.4352, 0.3120, 0.3095, 0.2990, 0.3060, 0.3943],
        [0.3861, 0.2918, 0.4081, 0.3147, 0.1554, 0.3744, 0.4438, 0.3677, 0.1418,
         0.3292, 0.4742, 0.2925, 0.4866, 0.1991, 0.5119, 0.3207]]).T

In [10]:
Mp[y_batch,range(Mp.size(1))]

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [9], [3]